In [628]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf


In [629]:
import contextlib
import sys, os
import warnings
warnings.filterwarnings('ignore')


@contextlib.contextmanager
def suppress_output():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr


In [630]:
import yfinance as yf

def download_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data['Date'] = data.index
    data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
    return data


In [631]:
def moving_average_strategy(data, short_window, long_window):
    data['short_mavg'] = data['Close'].rolling(window=short_window).mean()
    data['long_mavg'] = data['Close'].rolling(window=long_window).mean()
    data['signal'] = 0
    data['signal'][short_window:] = np.where(data['short_mavg'][short_window:] > data['long_mavg'][short_window:], 1, 0)
    data['positions'] = data['signal'].diff()
    return data


def MACD(data, short_window, long_window, signal_window):
    short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal_line = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal_line

def macd_strategy(data, short_window, long_window, signal_window):
    data['macd'], data['signal_line'] = MACD(data, short_window, long_window, signal_window)
    data['signal'] = 0
    data['signal'][short_window:] = np.where(data['macd'][short_window:] > data['signal_line'][short_window:], 1, 0)
    data['positions'] = data['signal'].diff()
    return data


In [632]:
def backtest(data, initial_capital):
    positions = pd.DataFrame(index=data.index).fillna(0.0)
    positions['Stock'] = 0.0  # Initialize the 'Stock' column with 0.0
    portfolio = pd.DataFrame(index=data.index).fillna(0.0)

    # Initial cash
    portfolio['cash'] = initial_capital
    portfolio['holdings'] = 0.0
    portfolio['total'] = initial_capital
    portfolio['returns'] = 0.0

    for i in range(1, len(data)):
        if data['signal'].iloc[i] == 1 and portfolio['cash'].iloc[i-1] >= data['Close'].iloc[i] * 100:
            # Buy signal and enough cash available
            positions['Stock'].iloc[i] = positions['Stock'].iloc[i-1] + 100
            portfolio['cash'].iloc[i] = portfolio['cash'].iloc[i-1] - data['Close'].iloc[i] * 100
        elif data['signal'].iloc[i] == 0 and positions['Stock'].iloc[i-1] >= 100:
            # Sell signal and enough stock available
            positions['Stock'].iloc[i] = positions['Stock'].iloc[i-1] - 100
            portfolio['cash'].iloc[i] = portfolio['cash'].iloc[i-1] + data['Close'].iloc[i] * 100
        else:
            # No trade, carry forward previous positions and cash
            positions['Stock'].iloc[i] = positions['Stock'].iloc[i-1]
            portfolio['cash'].iloc[i] = portfolio['cash'].iloc[i-1]

        # Calculate holdings
        portfolio['holdings'].iloc[i] = positions['Stock'].iloc[i] * data['Close'].iloc[i]
        portfolio['total'].iloc[i] = portfolio['cash'].iloc[i] + portfolio['holdings'].iloc[i]
        portfolio['returns'].iloc[i] = portfolio['total'].pct_change().iloc[i]

    return portfolio


In [633]:
import plotly.graph_objects as go

def generate_report(portfolio, initial_capital):
    total_return = portfolio['total'][-1] / initial_capital - 1
    annualized_return = (1 + total_return) ** (252 / len(portfolio)) - 1
    annualized_volatility = portfolio['returns'].std() * np.sqrt(252)
    sharpe_ratio = annualized_return / annualized_volatility

    # Print the statistics
    print(f'Total Return: {total_return:.2%}')
    print(f'Annualized Return: {annualized_return:.2%}')
    print(f'Annualized Volatility: {annualized_volatility:.2%}')
    print(f'Sharpe Ratio: {sharpe_ratio:.2f}')

    # Create the plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=portfolio.index, y=portfolio['total'], mode='lines', name='Total Portfolio Value'))
    fig.update_layout(title='Equity Curve',
                    xaxis_title='Date',
                    yaxis_title='Portfolio Value',
                    template='plotly_dark')

    fig.show()


In [634]:
def report():
    def backtest(data, initial_capital):
        positions = pd.DataFrame(index=data.index).fillna(0.0)
        positions['Stock'] = 0.0  # Initialize the 'Stock' column with 0.0
        portfolio = pd.DataFrame(index=data.index).fillna(0.0)

        # Initial cash
        portfolio['cash'] = initial_capital
        portfolio['holdings'] = 0.0
        portfolio['total'] = initial_capital
        portfolio['returns'] = 0.0

        # Track buy dates
        buy_dates = []

        for i in range(1, len(data)):
            if data['positions'].iloc[i] == 1:  # Checking for a buy signal
                if portfolio['cash'].iloc[i-1] >= data['Close'].iloc[i] * 100:
                    positions['Stock'].iloc[i] = positions['Stock'].iloc[i-1] + 100
                    portfolio['cash'].iloc[i] = portfolio['cash'].iloc[i-1] - data['Close'].iloc[i] * 100
                    buy_dates.append(data.index[i])  # Store buy date
                else:
                    positions['Stock'].iloc[i] = positions['Stock'].iloc[i-1]
                    portfolio['cash'].iloc[i] = portfolio['cash'].iloc[i-1]
            elif data['positions'].iloc[i] == -1:  # Checking for a sell signal
                if positions['Stock'].iloc[i-1] >= 100:
                    positions['Stock'].iloc[i] = positions['Stock'].iloc[i-1] - 100
                    portfolio['cash'].iloc[i] = portfolio['cash'].iloc[i-1] + data['Close'].iloc[i] * 100
                else:
                    positions['Stock'].iloc[i] = positions['Stock'].iloc[i-1]
                    portfolio['cash'].iloc[i] = portfolio['cash'].iloc[i-1]
            else:
                # No trade, carry forward previous positions and cash
                positions['Stock'].iloc[i] = positions['Stock'].iloc[i-1]
                portfolio['cash'].iloc[i] = portfolio['cash'].iloc[i-1]

            portfolio['holdings'].iloc[i] = positions['Stock'].iloc[i] * data['Close'].iloc[i]
            portfolio['total'].iloc[i] = portfolio['cash'].iloc[i] + portfolio['holdings'].iloc[i]
            portfolio['returns'].iloc[i] = portfolio['total'].pct_change().iloc[i]

        return portfolio, buy_dates


    with suppress_output():
        initial_capital = 100000.0
        portfolio, buy_dates = backtest(data, initial_capital)
    print("Buy Dates:")
    for date in buy_dates:
        print(date.strftime("%Y-%m-%d"))

    generate_report(portfolio, initial_capital)

In [635]:
# Step 1: Download data
data = download_data('COCHINSHIP.NS', '2024-01-01', '2024-07-07')

# Step 2: Apply a strategy and suppress output
with suppress_output():
    data = moving_average_strategy(data, short_window=40, long_window=100)

# Step 3: Backtest and suppress output
with suppress_output():
    initial_capital = 100000.0
    portfolio = backtest(data, initial_capital)

# Step 4: Generate report
report()


[*********************100%%**********************]  1 of 1 completed

Buy Dates:
Total Return: 0.00%
Annualized Return: 0.00%
Annualized Volatility: 0.00%
Sharpe Ratio: nan


In [636]:
# Step 1: Download data
data = download_data('COCHINSHIP.NS', '2024-01-01', '2024-07-07')

# Step 2: Apply a strategy and suppress output
with suppress_output():
    data = macd_strategy(data, short_window=12, long_window=26, signal_window=9)

# Step 3: Backtest and suppress output
with suppress_output():
    initial_capital = 100000.0
    portfolio = backtest(data, initial_capital)

# Step 4: Generate report
report()


[*********************100%%**********************]  1 of 1 completed

Buy Dates:
2024-01-17
2024-03-19
Total Return: 39.09%
Annualized Return: 94.47%
Annualized Volatility: 29.67%
Sharpe Ratio: 3.18


In [637]:
def combined_strategy(data, short_window=40, long_window=100, rsi_period=14):
    # Moving averages
    data['short_mavg'] = data['Close'].rolling(window=short_window).mean()
    data['long_mavg'] = data['Close'].rolling(window=long_window).mean()
    
    # RSI calculation
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=rsi_period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=rsi_period).mean()

    RS = gain / loss
    data['RSI'] = 100 - (100 / (1 + RS))

    # Generate signals based on MA and RSI
    data['signal'] = 0
    data.loc[(data['short_mavg'] > data['long_mavg']) & (data['RSI'] < 30), 'signal'] = 1
    data.loc[(data['short_mavg'] < data['long_mavg']) & (data['RSI'] > 70), 'signal'] = 0
    data['positions'] = data['signal'].diff()
    
    return data

# Use the strategy
with suppress_output():
    data = combined_strategy(data)

In [638]:
# Step 1: Download data
data = download_data('COCHINSHIP.NS', '2024-01-01', '2024-07-07')

# Step 2: Apply a strategy and suppress output
with suppress_output():
    data = combined_strategy(data)

# Step 3: Backtest and suppress output
with suppress_output():
    initial_capital = 100000.0
    portfolio = backtest(data, initial_capital)

# Step 4: Generate report
report()


[*********************100%%**********************]  1 of 1 completed

Buy Dates:
Total Return: 0.00%
Annualized Return: 0.00%
Annualized Volatility: 0.00%
Sharpe Ratio: nan


In [639]:
def breakout_strategy(data, window=30):
    
    data['20_day_high'] = data['Close'].rolling(window=window).max()
    data['20_day_low'] = data['Close'].rolling(window=window).min()
    data['signal'] = 0

    # Buy signal:
    data.loc[data['Close'] > data['20_day_high'].shift(1), 'signal'] = 1

    # Sell signal:
    #data.loc[data['Close'] < data['20_day_low'].shift(1), 'signal'] = -1
    
    data['positions'] = data['signal'].diff()
    return data

In [640]:
# Step 1: Download data
data = download_data('COCHINSHIP.NS', '2024-01-01', '2024-07-07')

# Step 2: Apply a strategy and suppress output
with suppress_output():
    data = breakout_strategy(data)

# Step 3: Backtest and suppress output
with suppress_output():
    initial_capital = 100000.0
    portfolio = backtest(data, initial_capital)

# Step 4: Generate report
report()

[*********************100%%**********************]  1 of 1 completed


Buy Dates:
2024-04-01
2024-04-05
Total Return: 15.01%
Annualized Return: 32.56%
Annualized Volatility: 14.92%
Sharpe Ratio: 2.18


In [641]:
def momentum_trading_strategy(data, momentum_window=7, hold_period=7):
    data['momentum'] = data['Close'].pct_change(periods=momentum_window)
    data['signal'] = 0
    data.loc[data['momentum'] > 0, 'signal'] = 1
    data['signal'] = data['signal'].shift(hold_period).fillna(0)
    data['positions'] = data['signal'].diff()
    return data

In [642]:
data = download_data('COCHINSHIP.NS', '2024-01-01', '2024-07-07')
with suppress_output():
    data = momentum_trading_strategy(data)
with suppress_output():
    initial_capital = 100000.0
    portfolio = backtest(data, initial_capital)
report()

[*********************100%%**********************]  1 of 1 completed


Buy Dates:
2024-01-19
2024-03-01
2024-03-05
2024-03-07
2024-03-19
2024-03-27
2024-04-10
2024-04-26
2024-05-03
Total Return: 47.39%
Annualized Return: 118.61%
Annualized Volatility: 44.53%
Sharpe Ratio: 2.66


In [643]:
import yfinance as yf
import pandas as pd

def download_data(ticker):
    return yf.download(ticker, start="2024-01-01", end="2024-07-07")

def momentum_trading_strategy(data, momentum_window=90):
    data['momentum'] = data['Close'].pct_change(periods=momentum_window)
    data['signal'] = (data['momentum'] > 0).astype(int)
    return data

def find_momentum_stocks(stock_list, momentum_window=90):
    momentum_stocks = []
    for ticker in stock_list:
        data = download_data(ticker)
        data = momentum_trading_strategy(data, momentum_window)
        if data['signal'].iloc[-1] == 1: 
            momentum_stocks.append(ticker)
    return momentum_stocks

# List of stock tickers to analyze
stock_list = ['ANGELONE.NS', 'GRSE.NS']

# Find stocks with positive momentum
momentum_stocks = find_momentum_stocks(stock_list)
print("Stocks with Positive Momentum:", momentum_stocks)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Stocks with Positive Momentum: ['GRSE.NS']
